In [5]:
import pandas as pd
import os

os.chdir(r"D:\Cours\annee3\Prononciation\cnam-athon-pronunciation-main")

In [14]:
df = pd.read_csv(r"data\dictionary\1757_Buchanan-J.csv", delimiter=";", encoding="utf8")

In [15]:
df.head()

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId
0,3,aʹbăcus,NaN,NaN,NaN,n.,2.0,19
1,2,ăbăʹft,NaN,NaN,NaN,p.,3.0,19
2,3,ăbaiʹſănce,F.,NaN,NaN,n.,4.0,19
3,3,ābăʹndon,F.,NaN,NaN,v.,5.0,19
4,4,ābăʹndŏned,NaN,NaN,NaN,a.,6.0,19


In [16]:
# définir une fonction qui retourne quelque chose
def sizePron(strPron):
    #retourner le nombre de caractère de la prononciation
    
    return len(strPron)

In [17]:
df['sizePron'] = df.apply(lambda x: sizePron(x['pron']),1)

In [18]:
df.sample(5)

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,sizePron
13141,1,ſtrait,NaN,NaN,NaN,n.,30.0,204,6
9703,3,păʹrădīſe,NaN,NaN,NaN,n.,39.0,156,9
1615,2,bŭʹdget,F.,NaN,NaN,n.,70.0,41,7
13839,1,thrŭms,NaN,NaN,NaN,n.,12.0,214,6
7688,1,launch,NaN,NaN,NaN,v.,26.0,128,6


In [19]:
pronChar = pd.read_csv(r"data\dictionary\Buchanan_PronChar_counts.csv", delimiter=";", encoding="utf8")

In [20]:
pronChar.sample(5)

,char,occ,graph
72,ï,1,i
53,(,28,NaN
56,а,2,a
30,p,3775,p
31,g,2604,g


In [92]:
def transcrire(mot):
    cheminDuFichierCSV = r"data/dictionary/Buchanan_PronChar_counts.csv"
    pronchar = pd.read_csv(cheminDuFichierCSV, delimiter=";", encoding="utf8")
    TruePron = []
    transfo = ''
    for lettre in mot:
        if (pronchar[pronchar['char']== lettre]['graph'].isnull().values[0]):
            transfo = ''
        else: transfo = pronchar[pronchar['char']== lettre]['graph'].values[0]
        TruePron.append(transfo)
    TruePron = ''.join(TruePron)
    return TruePron

In [22]:
df["transcription"] = df.apply(lambda x: transcrire(x['pron']),1)

In [23]:
df.sample(5)

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,sizePron,transcription
14035,3,trănsăʹctor,NaN,NaN,NaN,n.,64.0,216,11,transactor
3595,4,dīăgnŏʹſtic,G.,NaN,NaN,n.,49.0,69,11,diagnostic
2155,4,c_h_īrŏʹgrăpher,G.,NaN,NaN,n.,37.0,49,15,chirographer
6032,2,hāʹlo,G.,NaN,NaN,n.,42.0,103,5,halo
3610,4,dīăʹphăn_o_us,G.,NaN,NaN,a.,64.0,69,13,diaphanous


In [24]:
cheminDuFichierCSV = r"data\dataframe\BigiRichSample.csv"
bigi = pd.read_csv(cheminDuFichierCSV, delimiter=";", encoding="utf8")
merged = df.merge(bigi, how='left',left_on='transcription', right_on='hwd')

In [49]:
merged = merged.drop("hwd", axis=1)
merged[merged["transcription"] == "architect"]

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,sizePron,transcription,pronSampa,ipa
690,3,ăʹrc_h_ĭtect,G.,NaN,NaN,n.,8.0,29,12,architect,A r\ k @ t E k t,ɑɹkətɛkt


In [50]:
#export en CSV pour conserver la colonne et l'utiliser plus tard
merged.to_csv(r"data/dictionary/Buchanan_enrichi.csv",index=False, encoding='utf-8')

In [63]:
explain = merged
explain['ipa'] = explain['ipa'].fillna(0)

In [72]:
print("taille de bigi:", bigi.shape[0])
print("taille du dictionnaire :",explain.shape[0])
print("taille du dictionnaire non expliqué par ipa:",explain[explain["ipa"] == 0].shape[0], "vs taille ciblée:", explain.shape[0]-bigi.shape[0])

taille de bigi: 1000
taille du dictionnaire : 15800
taille du dictionnaire non expliqué par ipa: 15723 vs taille ciblée: 14800


L'explication à cette différence peut être diverse : le mot peut tout simplement ne pas exister dans Buchanan, ou bien être écrit différemment. Parfois, la transcription a aussi des défauts comme ci-dessous avec "water" où la prononciation "wauter" est entre parenthèse, ce qui rend la transciption fausse. Il faudrait corriger ces cas-là pour améliorer immédiatement la jointure entre Buchanan et Bigi.

In [81]:
merged[merged['transcription']=='water wauter']

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,sizePron,transcription,pronSampa,ipa
15447,2,waʹt_e_r (wauter),NaN,NaN,Aide à la {pron}.,n.,45.0,236,17,water wauter,NaN,0


Ci-dessous la fonction corrigée pour éviter de garder les éléments entre parenthèses dans Buchanan.

In [90]:
def transcrireCorrige(mot):
    cheminDuFichierCSV = r"data/dictionary/Buchanan_PronChar_counts.csv"
    pronchar = pd.read_csv(cheminDuFichierCSV, delimiter=";", encoding="utf8")
    TruePron = []
    transfo = ''
    for lettre in mot:
        if lettre == '(':
            break
        if (pronchar[pronchar['char']== lettre]['graph'].isnull().values[0]):
            transfo = ''
        else: transfo = pronchar[pronchar['char']== lettre]['graph'].values[0]
        TruePron.append(transfo)
    TruePron = ''.join(TruePron)
    return TruePron.strip()

In [100]:
df2 = pd.read_csv(r"data\dictionary\1757_Buchanan-J.csv", delimiter=";", encoding="utf8")
df2['sizePron'] = df2.apply(lambda x: sizePron(x['pron']),1)
df2["transcription"] = df2.apply(lambda x: transcrireCorrige(x['pron']),1)
merged2 = df2.merge(bigi, how='left',left_on='transcription', right_on='hwd')
merged2.to_csv(r"data/dictionary/Buchanan_enrichi.csv",index=False, encoding='utf-8')

In [102]:
merged2[merged2["transcription"] == "water"]

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,sizePron,transcription,hwd,pronSampa,ipa
15447,2,waʹt_e_r (wauter),NaN,NaN,Aide à la {pron}.,n.,45.0,236,17,water,NaN,NaN,NaN


In [104]:
explain2 = merged2
explain2['ipa'] = explain2['ipa'].fillna(0)

In [106]:
print("taille de bigi:", bigi.shape[0])
print("taille du dictionnaire :",explain2.shape[0])
print("taille du dictionnaire non expliqué par ipa:",explain2[explain2["ipa"] == 0].shape[0], "vs taille ciblée:", explain2.shape[0]-bigi.shape[0])

taille de bigi: 1000
taille du dictionnaire : 15800
taille du dictionnaire non expliqué par ipa: 15723 vs taille ciblée: 14800


Finalement on voit que cette correction n'a pas eu l'effet escompté, ce n'est visiblement pas ça qui posait problème. C'est plus vraisemblablement que les 2 dictionnaires ont très peu de mots en commun.